
# OpenRouter x Python 


Objectif : appeler **OpenRouter** depuis Python en lisant la clé **depuis `.env`** (variable `OPENROUTER_API_KEY`).  
Nous montrons :
1. Chargement des variables d'environnement
2. Appel **non-streaming** (simple)
3. Appel **streaming** (ligne par ligne)

> Prérequis : créer un fichier `.env` à la racine de votre projet contenant :  
> `OPENROUTER_API_KEY=sk-...`


## 0) Dépendances

In [ ]:

# Décommentez si besoin
# !pip install requests python-dotenv -q


## 1) Charger la clé API depuis `.env`

In [5]:

import os
from dotenv import load_dotenv

# Charge .env
load_dotenv()

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
assert OPENROUTER_API_KEY, "La variable d'environnement OPENROUTER_API_KEY est absente. Ajoutez-la dans votre .env"
print("Clé détectée (tronquée) :", OPENROUTER_API_KEY[:8] + "..." if OPENROUTER_API_KEY else "Aucune")


Clé détectée (tronquée) : sk-or-v1...


## 2) Paramètres simples

In [13]:

from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1"

# Choisissez un modèle auquel votre compte a accès.
# Exemples fréquents (selon disponibilités) :
# - "meta-llama/llama-3.1-70b-instruct"
# - "anthropic/claude-3.5-sonnet"
# - "openai/gpt-4o-mini"
# - "mistralai/mixtral-8x7b-instruct"
MODEL = "anthropic/claude-3.5-sonnet"  # <— changez si nécessaire

llm = ChatOpenAI(
    model=MODEL,
    api_key=OPENROUTER_API_KEY,
    base_url=OPENROUTER_BASE_URL,
    temperature=0.2,
)

print("Modèle :", MODEL)


Modèle : anthropic/claude-3.5-sonnet


## 3) Appel 


In [14]:

payload = {
    "model": MODEL,
    "messages": [
        {"role": "system", "content": "Tu es un assistant concis et pédagogique."},
        {"role": "user", "content": "Explique en 2 phrases ce qu'est un LLM et donne un cas d'usage concret."}
    ],
    "stream": False  # on veut une seule réponse JSON
}

resp = llm.invoke([
    SystemMessage(content="Tu es un assistant concis et pédagogique."),
    HumanMessage(content="Explique en 2 phrases ce qu'est un LLM et donne un cas d'usage concret."),
])
print("\nRéponse :\n", resp.content)


APIStatusError: Error code: 402 - {'error': {'message': 'This request requires more credits, or fewer max_tokens. You requested up to 8192 tokens, but can only afford 2664. To increase, visit https://openrouter.ai/settings/credits and upgrade to a paid account', 'code': 402, 'metadata': {'provider_name': None}}, 'user_id': 'user_2uYzUCT7Pg5cO0VFuRPyiAHOLJ4'}

## 4) Appel **streaming** (ligne par ligne)

In [ ]:

import json

payload_stream = {
    "model": MODEL,
    "messages": [
        {"role": "system", "content": "Tu es un assistant concis et pédagogique."},
        {"role": "user", "content": "Donne 4 bonnes pratiques pour écrire un prompt efficace."}
    ],
    "stream": True  # renverra un flux d'events JSON (SSE/NDJSON)
}

for chunk in llm.stream([
    SystemMessage(content="Tu es un assistant concis et pédagogique."),
    HumanMessage(content="Donne 4 bonnes pratiques pour écrire un prompt efficace."),
]):
    if chunk.content:
        print(chunk.content, end="")
print("\n— Fin du streaming —")


Voici quatre bonnes pratiques pour écrire un prompt efficace:

1.  **Soyez précis** : un prompt clair doit être direct et spécifique, indiquant exactement ce que l'utilisateur doit faire ou réponse attendue. Évitez les formulations vagues ou trop larges.
2.  **Utilisez un langage simple et concis** : employez des termes courants et un ton naturel, vous donnez ainsi plus de probabilité à ce que l'utilisateur comprenne le message sans erreur.
3.  **Évitez les ambiguïtés ou les questions pièges** : assurez-vous que le prompt n’est pas susceptible d’être interprété de manière multiple afin que l'utilisateur donne la bonne réponse. Pour ce faire, vous pouvez auss modo atteindre quelques explications ou indications supplémentaires. 
4.  **Faites des essais** : testez le prompt avec différents utilisateurs et ajustez-le en fonction de leurs réponses pour en améliorer l'efficacité.
— Fin du streaming —


APIStatusError: Error code: 402 - {'error': {'message': 'This request requires more credits, or fewer max_tokens. You requested up to 8192 tokens, but can only afford 2664. To increase, visit https://openrouter.ai/settings/credits and upgrade to a paid account', 'code': 402, 'metadata': {'provider_name': None}}, 'user_id': 'user_2uYzUCT7Pg5cO0VFuRPyiAHOLJ4'}